# C09. 魔法方法

-   构造函数：`__init__`，对象被构造时调用
-   析构函数：`__del__`，对象被销毁时(作为垃圾被收集)前被调用，因为垃圾收集时间由系统决定，因此函数的行为不可预知，建议尽量不使用这个函数。

## 9.1 Python 2

Python 3 不需要显式地继承 object 或者 将 `__metaclass__` 设置为 type。所有的类都将隐式地继承 object。如果没有指定超类，将直接继承它，否则将间接地继承它。

## 9.2 构造函数

构造函数 `__init__` 将在对象创建后自动被调用。

In [2]:
class FooBar:
    def __init__(self, value=42):
        self.somevar=value

f1=FooBar()
print(f1.somevar)

f2=FooBar("This is a constructor argument.")
print(f2.somevar)

42
This is a constructor argument.


### 9.2.1 重写函数

In [3]:
class A:
    def hello(self):
        print("Hello, I'm A.")
class B(A):
    pass

a=A()
b=B()
a.hello()
b.hello()

Hello, I'm A.
Hello, I'm A.


In [4]:
class B(A):
    def hello(self):
        print("Hello, I'm B.")
b=B()
b.hello()

Hello, I'm B.


In [5]:
class Bird:
    def __init__(self):
        self.hungry=True
    def eat(self):
        if self.hungry:
            print("Aaaah...")
            self.hungry=False
        else:
            print("No, thks!")
b=Bird()
b.eat()
b.eat()            

Aaaah...
No, thks!


In [8]:
class SongBird(Bird):
    def __init__(self):
        self.sound='Squawk!'
    def sing(self):
        print(self.sound)

sb=SongBird()
sb.sing()
# sb.eat()  # 构造函数重写后，未调用超类的构造函数，导致超类的属性未被定义

Squawk!


### 9.2.2 调用超类的构造函数：通过类名调用

In [9]:
class SongBird(Bird):
    def __init__(self):
        Bird.__init__(self)
        self.sound='Squawk!'
    def sing(self):
        print(self.sound)

sb=SongBird()
sb.sing()
sb.eat()

Squawk!
Aaaah...


### 9.2.3 调用超类的构造函数：通过 super 函数

super 函数：可以自动寻找子类的父类，避免硬绑定。

In [10]:
class SongBird(Bird):
    def __init__(self):
        super().__init__()
        self.sound='Squawk!'
    def sing(self):
        print(self.sound)

sb=SongBird()
sb.sing()
sb.eat()
sb.eat()

Squawk!
Aaaah...
No, thks!


## 9.3 元素访问

协议：指的是规范行为的规则，类似于接口。协议指定应该实现哪些方法以及这些方法应该做些什么。在 Python 中，多态仅仅基于对象的行为，而不像其他语言那样要求对象基于特定的类或者实现特定的接口。

### 9.3.1 基本协议：序列和映射

序列和映射都是元素的集合，要实现它们的基本协议，不可变对象需要实现 2 个方法，可变对象需要实现 4 个方法。
-   `__len__(self)`：返回集合包含的项数，序列即元素个数，映射即键-值对的数目。如果返回零，对象在布尔上下文中将被视为假(就像空的列表、元组、字符-   串、字典)。
-   `__getitem__(self,key)`：返回与指定键相关联的值，序列中键为0到(n-1)的整数，n为序列的长度；映射中键可以为任何类型。
-   `__setitem__(self,key,value)`：通过与键相关联的方式存储值，满足 `__getitem__` 来获取。仅用于可变对象
-   `__delitem__(self,key)`：在对对象的组成部分使用 `__del__` 语句时被调用，删除与键相关联的值。仅用于可变对象

说明：
-   对于序列，如果键为负整数，则从尾往前数
-   如果键的类型不合适，可能引发 TypeError 异常
-   对于序列，如果索引的类型是正确的，但是超出了允许的范围，将引发 IndexError 异常

In [7]:
# 创建无穷序列
def check_index(key):
    if not isinstance(key,int):raise TypeError
    if key<0:raise IndexError

class ArithmeticSequence:
    def __init__(self,start=0,step=1):
        self.start=start
        self.step=step
        self.changed={}
    def __getitem__(self,key):
        check_index(key)
        try: return self.changed[key]
        except KeyError:
            return self.start+key*self.step
    def __setitem__(self,key,value):
        check_index(key)
        self.changed[key]=value

s=ArithmeticSequence(1,2)
print(s[4])
s[4]=2
print(s[4])
print(s[5])
# del s[4]  # 没有实现 __delitem__ 函数，禁止删除元素
# print(s['four'])  # 错误的索引类型
# print(s[-5])      # 错误的索引位置

9
2
11


### 9.3.2 从系统的序列类(list, dict, str)中派生

In [9]:
# 带访问计数器的列表
class CounterList(list):
    def __init__(self,*args):
        super().__init__(*args)
        self.counter=0
    def __getitem__(self,index):
        self.counter+=1
        return super(CounterList,self).__getitem__(index)

c1=CounterList(range(10))
print("c1=",c1)
c1.reverse()
print("c1=",c1)
del c1[3:6]
print("c1=",c1)
# c1.__getitem__ 被调用的次数
print("c1.counter=",c1.counter)
print("c1[4]+c1[2]=",c1[4]+c1[2])
print("c1.counter=",c1.counter)

c1= [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
c1= [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
c1= [9, 8, 7, 3, 2, 1, 0]
c1.counter= 0
c1[4]+c1[2]= 9
c1.counter= 2


## 9.4 其他的魔法方法

参阅《Python Reference Manaul》中的「Special method names」节。

## 9.5 特性

In [3]:
class Rectangle:
    def __init__(self):
        self.width=0
        self.height=0
    def set_size(self, size):
        self.width, self.height=size
    def get_size(self):
        return self.width, self.height

r=Rectangle()
r.width=10
r.height=5
print("r.get_size()=",r.get_size())
r.set_size((150,100))
print("r.width=",r.width)

r.get_size()= (10, 5)
r.width= 150


### 9.5.1 property 函数

通过存取方法定义的属性通常称为特征(property)。

说明：property 本质不是函数，而是一个类。它的实例包含一些魔法方法(__get__, __set__, __delete__)，这些魔法方法定义了所谓的描述符协议。只要对象实现了这些方法中的任何一个，它就是一个描述符。

描述符的说明参考 [Descriptor HowTo Guide](https://docs.python.org/3/howto/descriptor.html)

In [4]:
class Rectangle:
    def __init__(self):
        self.width=0
        self.height=0
    def set_size(self, size):
        self.width, self.height=size
    def get_size(self):
        return self.width, self.height
    size=property(get_size,set_size)

r=Rectangle()
r.width=10
r.height=5
print("r.size=",r.size)
r.size=150,100
print("r.width=",r.width)

r.size= (10, 5)
r.width= 150


### 9.5.2 静态方法 与 类方法

静态方法的定义中没有参数 self，可以直接通过类来调用。
类方法的定义中包含参数 self，可以通过类调用，也可以通过对象调用。

In [6]:
class MyClass:
    @staticmethod
    def smeth():
        print("This is a static method!")
    
    @classmethod
    def cmeth(cls):
        print("This is a class method!")

MyClass.smeth()
MyClass.cmeth()
m=MyClass()
m.cmeth()

This is a static method!
This is a class method!
This is a class method!


### 9.5.3 控制对象属性访问的魔法方法：`__getattr__`, `__setattr__`

-   `__getattribute__(self,name)`：在属性被访问时自动调用
-   `__getattr__(self,name)`：在属性被访问而对象没有这样的属性时自动调用
-   `__setattr__(self,name,value)`：在给属性赋值时自动调用
-   `__delattr__(self,name)`：在删除属性时自动调用

In [7]:
class Rectangle:
    def __init__(self):
        self.width=0
        self.height=0
    def __setattr__(self,name,value):
        if name=='size':
            self.width,self.height=value
        else:
            # 使用 __dict__ 赋值，而不直接赋值是避免出现再次调用 __setattr__，导致无限循环
            # 使用 __dict__ 赋值时，也要避免 __getattribute__ 拦截所有访问导致无限循环，
            self.__dict__[name]=value
    def __getattr__(self,name):
        if name=='size':
            return self.width,self.height
        else:
            raise AttributeError()

r=Rectangle()
r.width=10
r.height=5
print("r.size=",r.size)
r.size=150,100
print("r.width=",r.width)        


r.size= (10, 5)
r.width= 150


## 9.6 迭代器

### 9.6.1 迭代器协议

迭代(iter)：意味着重复多次，就像循环一样。

注意：实现了 `__iter__` 的对象是可迭代的，实现了 `__next__` 的对象是迭代器，方法 `__iter__` 返回迭代器。

In [8]:
class Fibs:
    def __init__(self):
        self.a=0
        self.b=1
    def __next__(self):
        self.a, self.b=self.b,self.a+self.b
        return self.a
    def __iter__(self):
        return self

fibs=Fibs()
for f in fibs:
    if f>1000:
        print(f)
        break

1597


In [12]:
# 使用内置函数 `iter` 也可以获得迭代器
for i in [1,5,7]:
    print(i)

for it in iter([1,5,7]):
    print(it)

1
5
7
1
5
7


### 9.6.2 基于迭代器创建序列

In [13]:
class TestIterator:
    value=0
    def __next__(self):
        self.value+=1
        if self.value>10:raise StopIteration
        return self.value
    def __iter__(self):
        return self

ti=TestIterator()
list(ti)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

## 9.7 生成器

生成器是使用普通函数语法定义的迭代器

### 9.7.1 创建生成器

In [14]:
# 包含 yield 语句的函数都被称为生成器
# 每次使用 yield 生成一个值后，函数都将被冻结，并在此等待被重新唤醒，唤醒后继续执行
def flatten(nested):
    for sublist in nested:
        for element in sublist:
            yield element

nested=[[1,2],[3,4],[5]]
for num in flatten(nested):
    print(num)

1
2
3
4
5


### 9.7.2 递归式生成器

调用 flatten 时，有两种可能性：基线条件 和 递归条件。
在基线条件下，要求这个函数展开单个元素(例如一个数)

In [19]:
def flatten(nested):
    try:
        for sublist in nested:
            for element in flatten(sublist):
                yield element
    except TypeError:
        yield nested

print(list(flatten([[[1],2],3,4,[5,[6,7]],8])))

print(list(flatten(1)))

# 对字符串迭代会导致无穷递归，因为字符串的第一个元素是一个长度为 1 的字符串，而长度为 1 的字符串的第一个元素就是字符串本身
# print(list(flatten(['foo',['bar',['baz']]])))

[1, 2, 3, 4, 5, 6, 7, 8]
[1]


In [20]:
def flatten(nested):
    try:
        # 如果是字符串对象，直接抛出 TypeError 从而避免迭代
        try: nested+''
        except TypeError: pass
        else: raise TypeError
        
        for sublist in nested:
            for element in flatten(sublist):
                yield element
    except TypeError: yield nested

print(list(flatten(['foo',['bar',['baz']]])))

['foo', 'bar', 'baz']


### 9.7.3 通用的生成器

生成器的函数 和 生成器的迭代器 通称为 生成器。

In [21]:
def simple_generator():
    yield 1

print(simple_generator)
print(simple_generator())
for i in simple_generator():
    print("i=",i)

<function simple_generator at 0x00000000079156A8>
<generator object simple_generator at 0x000000000953EDB0>
i= 1


### 9.7.4 生成器的方法

-   外部世界：可以访问生成器的方法 send，类似于 next，但是要接受一个参数(值为需要发送的「消息」，「消息」可以是任何对象)
-   生成器：在挂起的生成器内部， yield 可能用作表达式而不是语句。当生成器重新运行时，yield  返回一个值——通过 send 从外部世界发送的值。如果使用的是 next，yield 返回 None。
    -   throw 方法：用于在生成器中(yield 表达式处) 引发异常，调用时可以提供一个异常类型、一个可选值、一个 traceback 对象
    -   close 方法：用于停止生成器，调用时无需提供任何参数

In [23]:
def repeater(value):
    while True:
        new=(yield value)
        if new is not None: value=new
r=repeater(42)
print(next(r))
print(next(r))
print(r.send("Hello, world!"))        

42
42
Hello, world!


### 9.7.5 模拟生成器

In [25]:
# 使用普通函数模拟生成器
def flatten(nested):
    result=[]
    try:
        try: nested+''
        except TypeError: pass
        else: raise TypeError
        
        for sublist in nested:
            for element in flatten(sublist):
                result.append(element)
    except TypeError: result.append(nested)
    return result

print(list(flatten([[[1],2],3,4,[5,[6,7]],8])))
print(list(flatten(1)))
print(list(flatten(['foo',['bar',['baz']]])))

[1, 2, 3, 4, 5, 6, 7, 8]
[1]
['foo', 'bar', 'baz']


## 9.8 经典问题：八皇后问题

### 9.8.1 生成器回溯

-   不使用生成器时，算法需要通过额外的参数来传递部分结果，从而让递归能够接着往下计算。
-   使用生成器时，所有的递归调用都只需要生成其负责部分的结果，这种策略可以用来遍历图结构和树结构。

### 9.8.2 问题描述

将 8 个皇后放在棋盘上，要求是任何一个皇后都不能威胁到其他皇后，即任何两个皇后都不能吃掉对方。

这是一个典型的回溯问题：在棋盘的第一行尝试为第一个皇后选择一个位置，再在第二行尝试为第二个皇后选择一个位置，依次类推至无法为当前皇后找到合适位置时，就回溯到前一个皇后，并尝试为它选择另一个位置。最后，要么深度完成所有可能而失败，要么找到正确的答案。

### 9.8.3 状态表示

`state[0]=3`：表示第 1 行的皇后放在第 4 列。

### 9.8.4 检测冲突

要找出没有冲突的位置组合，首先定义冲突是什么？

In [2]:
def conflict(state, nextX):
    nextY=len(state)
    for i in range(nextY):
        if abs(state[i]-nextX) in (0,nextY-i):
            # 如果下一个皇后与当前皇后的 x 坐标相同，则发生冲突
            # 如果下一个皇后与当前皇后在左对角线(负值)或者右对角线(正值)时，则发生冲突
            return True

### 9.8.5基线条件


In [17]:
def queens(num,state):
    if len(state)==num-1:
        for pos in range(num):
            if not conflict(state,pos):
                yield pos
print(list(queens(3,(1,3))))
print(list(queens(4,(1,3,0))))
print(list(queens(5,(1,3,0,2))))
print(list(queens(6,(1,3,0,2,4))))

[0]
[2]
[4]
[]


### 9.8.6 递归条件

In [1]:
def queens(num=8, state=()):
    for pos in range(num):
        if not conflict(state,pos):
            if len(state)==num-1:
                # 返回的序列长度与棋盘大小一样时
                yield(pos,)
            else:
                for result in queens(num, state+(pos,)):
                    yield (pos,)+result

### 9.8.7 扫尾工作

In [5]:
def pretty_print(solution):
    def line(pos, length=len(solution)):
        return '. '*(pos)+'X '+'. '*(length-pos-1)
    for pos in solution:
        print(line(pos))

In [6]:
import random
pretty_print(random.choice(list(queens())))

. X . . . . . . 
. . . . X . . . 
. . . . . . X . 
X . . . . . . . 
. . X . . . . . 
. . . . . . . X 
. . . . . X . . 
. . . X . . . . 


## 9.9小结

-   新式类 和 旧式类：Python 2.2 开始引入新式类
-   魔法方法：Python 的特殊方法，名称以两个下划线开头和结尾。这些方法的功能各不相同，但是大都由 Python 在特定的情况下自动调用。
-   构造函数：对象创建后自动调用 `__init__`
-   重写：子类可以重写超类中的方法(以及其他任何属性)。调用被重写的超类方法，可以通过超类直接调用，也可以通过函数 `super` 调用
-   序列 和 映射：创建自定义的序列或者映射，可以实现序列或者映射指定的所有方法；也可以从 list 或者 dict 派生
-   迭代器：包含方法 `__next__` 的对象，可以用于迭代一组值。没有更多的值可供迭代时，方法将会引发 `StopIteration` 异常。可迭代的对象包含方法 `__iter__`，这个方法返回一个像序列一样可以用于 for 循环中的迭代器。
-   生成器：生成器的函数包含关键字 yield，函数在被调用时返回一个生成器，即一种特殊的迭代器。使用方法 `send`、`throw`、`close` 可以与活动的生成器交互。
-   八皇后问题：：著名的计算机科学问题，使用生成器解决它。